# Homework and bakeoff: Multi-domain sentiment

In [ ]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2023"

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cgpotts/cs224u/blob/main/hw_sentiment.ipynb)

If Colab is opened with this badge, please **save a copy to drive** (from the File menu) before running the notebook.

## Overview

This homework and associated bakeoff are devoted to supervised sentiment analysis in a ternary label setting (positive, negative, neutral). Your ultimate goal is to develop systems that can make accurate predictions in multiple domains.

The homework questions ask you to implement some baseline systems using DynaSent Round 1, DynaSent Round 2, and the Stanford Sentiment Treebank. The bakeoff challenge is to define a system that does well on the DynaSent test sets, the SST-3 test set, and a set of mystery examples that don't correspond to the DynaSent or SST-3 domains.

__Important methodological note:__ The DynaSent and SST-3 test sets are already publicly distributed, so we are counting on people not to cheat by developing their models on these test sets. You must do all your development without using these test sets at all, and then evaluate exactly once on the test set and turn in the results, with no further system tuning or additional runs. _Much of the scientific integrity of our field depends on people adhering to this honor code._

This notebook briefly introduces our three development datasets, states the homework questions, and then provides guidance on the original system and associated bakeoff entry.

## Set-up

In [1]:
try:
    # Sort of randomly chosen import to see whether the requirements
    # are met:
    import datasets
except ModuleNotFoundError:
    !git clone https://github.com/cgpotts/cs224u/
    !pip install -r cs224u/requirements.txt
    import sys
    sys.path.append("cs224u")

Cloning into 'cs224u'...
remote: Enumerating objects: 2409, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 2409 (delta 150), reused 127 (delta 119), pack-reused 2177 (from 3)
Receiving objects: 100% (2409/2409), 41.76 MiB | 12.84 MiB/s, done.
Resolving deltas: 100% (1467/1467), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.7/280.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.1/116.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.5/292.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from collections import defaultdict, Counter
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import torch

## Datasets

### DynaSent round 1

The DynaSent dataset of [Potts, Wu, et al. 2021](https://aclanthology.org/2021.acl-long.186/) is a ternary sentiment benchmark consisting of two rounds (so far). The dataset is available on [Hugging Face](https://huggingface.co/datasets/dynabench/dynasent).

For Round 1, the authors collected sentences from the [Yelp Academic Dataset](https://www.yelp.com/dataset) that fooled a top-performing sentiment model but were intuitive for humans. The model was used only to heuristically find the examples. Crowdworkers multiply-labeled all of them.

The round contains a lot of metadata that could be useful for developing sentiment models. We will focus on just the sentences and labels, but you are free to make use of this additional metadata in developing uour system.

In [3]:
dynasent_r1 = load_dataset("dynabench/dynasent", 'dynabench.dynasent.r1.all', trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

dynasent.py:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/80488 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3600 [00:00<?, ? examples/s]

In [4]:
dynasent_r1

DatasetDict({
    train: Dataset({
        features: ['id', 'hit_ids', 'sentence', 'indices_into_review_text', 'model_0_label', 'model_0_probs', 'text_id', 'review_id', 'review_rating', 'label_distribution', 'gold_label', 'metadata'],
        num_rows: 80488
    })
    validation: Dataset({
        features: ['id', 'hit_ids', 'sentence', 'indices_into_review_text', 'model_0_label', 'model_0_probs', 'text_id', 'review_id', 'review_rating', 'label_distribution', 'gold_label', 'metadata'],
        num_rows: 3600
    })
    test: Dataset({
        features: ['id', 'hit_ids', 'sentence', 'indices_into_review_text', 'model_0_label', 'model_0_probs', 'text_id', 'review_id', 'review_rating', 'label_distribution', 'gold_label', 'metadata'],
        num_rows: 3600
    })
})

Splits:

In [5]:
def print_label_dist(dataset, labelname='gold_label', splitnames=('train', 'validation')):
    for splitname in splitnames:
        print(splitname)
        dist = sorted(Counter(dataset[splitname][labelname]).items())
        for k, v in dist:
            print(f"\t{k:>14s}: {v}")

In [6]:
print_label_dist(dynasent_r1)

train
	      negative: 14021
	       neutral: 45076
	      positive: 21391
validation
	      negative: 1200
	       neutral: 1200
	      positive: 1200


### DynaSent round 2

DynaSent Round 2 was created using different methods than Round 1. For Round 2, crowdworkers edited sentences from the Yelp Academic Dataset seeking to achieve a particular sentiment goal (e.g., expressing a positive sentiment) while fooling a top-performing model. This work was done on the [Dynabench](https://dynabench.org) platform. The hope is that this directly adversarial goal will lead to examples that are very hard for present-day models but intuitive for humans. All the examples were multiply-labeled by separate annotators.

In [7]:
dynasent_r2 = load_dataset("dynabench/dynasent", 'dynabench.dynasent.r2.all', trust_remote_code=True)

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/13065 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/720 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/720 [00:00<?, ? examples/s]

In [8]:
print_label_dist(dynasent_r2)

train
	      negative: 4579
	       neutral: 2448
	      positive: 6038
validation
	      negative: 240
	       neutral: 240
	      positive: 240


### Stanford Sentiment Treebank

The [Stanford Sentiment Treebank (SST)](http://nlp.stanford.edu/sentiment/) of [Socher et al. 2013](https://aclanthology.org/D13-1170/) is a widely-used resource for evaluating supervised models. It consists of sentences from Rotten Tomatoes Movie Reviews (see [Pang and Lee's project page](https://www.cs.cornell.edu/home/llee/papers/pang-lee-stars.home.html)). We will use the ternary version of the task (SST-3).

SST examples are special in that they are labeled at the phrase-level as well as the sentence level, which provides very extensive and detailed supervision for sentiment. We will use only the sentence-level labels for the homework, but you are free to use the phrase-level labels as well in designing your original system. (To do this, you will need to get the dataset from the above project page, since the Hugging Face SST-3 we are using does not include these labels.)

In [9]:
sst = load_dataset("SetFit/sst5", trust_remote_code=True)

README.md:   0%|          | 0.00/421 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

dev.jsonl:   0%|          | 0.00/171k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/343k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

In [10]:
sst

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
})

Out of the box, this is a five-way task:

In [11]:
print_label_dist(sst, labelname='label_text')

train
	      negative: 2218
	       neutral: 1624
	      positive: 2322
	 very negative: 1092
	 very positive: 1288
validation
	      negative: 289
	       neutral: 229
	      positive: 279
	 very negative: 139
	 very positive: 165


The above labels are not aligned with our ternary task, and the dataset distribution uses slightly different keys from those of DynaSent. The following code converts the dataset to SST-3 and also aligns the dataset keys:

In [12]:
def convert_sst_label(s):
    return s.split(" ")[-1]

In [13]:
for splitname in ('train', 'validation', 'test'):
    dist = [convert_sst_label(s) for s in sst[splitname]['label_text']]
    sst[splitname] = sst[splitname].add_column('gold_label', dist)
    sst[splitname] = sst[splitname].add_column('sentence', sst[splitname]['text'])

In [14]:
print_label_dist(sst)

train
	      negative: 3310
	       neutral: 1624
	      positive: 3610
validation
	      negative: 428
	       neutral: 229
	      positive: 444


## Question 1: Linear classifiers

Our first set of experiments will use simple linear classifiers with sparse representations derived from counting unigrams. These experiments will introduce some useful techniques and provide a baseline for original systems.

The tokenization scheme used by `unigrams_phi` is very basic and leads to unintuitive tokens with punctuation attached to them. Your task here is to complete `tweetgrams_phi`, which should lead to more intuitive results. The task is really just to use the NLTK [TweetTokenizer](https://www.nltk.org/api/nltk.tokenize.casual.html#nltk.tokenize.casual.TweetTokenizer) in place of the simple whitespace tokenization of `unigrams_phi` above.

In [15]:
def unigrams_phi(s):
    """The basis for a unigrams feature function.

    Downcases all tokens.

    Parameters
    ----------
    s : str
        The example to represent

    Returns
    -------
    Counter
        A map from tokens (str) to their counts in `text`

    """
    return Counter(s.lower().split())
# Your `tweetgrams_phi` should tokenize data according to this tokenizer from NLTK:
from nltk.tokenize import TweetTokenizer

def tweetgrams_phi(s, **kwargs):
    """The basis for a feature function using `TweetTokenizer`.

    Parameters
    ----------
    s : str
    kwargs : dict
        Passed to `TweetTokenizer`

    Returns
    -------
    Counter
        A map from tokens to their counts in `text`

    """
    #Instantiate the TweetTokenizer with the given arguments:
    tokenizer = TweetTokenizer(**kwargs)

    #Tokenize the input string and return the tokens as a Counter:
    return Counter(tokenizer.tokenize(s))



Here's a test you can use to check that your implementation is correct:

In [16]:
def test_tweetgrams_phi(func):
    examples = [
        (
            "Here's an example with an emoticon :)",
            Counter({'an': 2, "Here's": 1, 'example': 1, 'with': 1, 'emoticon': 1, ':)': 1})
        ),
        (
            "The URL is https://pytorch.org!",
            Counter({'The': 1, 'URL': 1, 'is': 1, 'https://pytorch.org': 1, '!': 1})
        )
    ]
    errcount = 0
    for ex, expected in examples:
        result = func(ex, preserve_case=True)
        if result != expected:
            errcount += 1
            print(f"Error for `{func.__name__}`: For input {ex}, "
                  f"expected {expected} but got {result}")
    caps_ex = "CAPS"
    caps_result = func(caps_ex, preserve_case=False)
    caps_expected = Counter({"caps": 1})
    if caps_result != caps_expected:
        errcount += 1
        print(f"Error for `{func.__name__}`: For input {caps_ex}, "
              f"expected {caps_expected} but got {caps_result}")
    if errcount == 0:
        print(f"All tests passed for `{func.__name__}`")

In [17]:
test_tweetgrams_phi(tweetgrams_phi)

All tests passed for `tweetgrams_phi`


### Task 2: Model training [1 point]

Your task is to complete `train_linear_model`:

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report

In [19]:
def train_linear_model(model, featfunc, train_dataset):
    """Train an sklearn classifier.

    Parameters
    ----------
    model : sklearn classifier model
    featfunc : func
        Maps strings to Counter instances
    train_dataset: dict
        Must have a key "sentence" containing strings that `featfunc`
        will process, and a key "gold_label" giving labels

    Returns
    -------
    tuple
        * A trained version of `model`
        * A fitted `vectorizer` for the train set

    """
     # Step 1: Featurize all the examples in `train_dataset['sentence']`
    feats = [featfunc(s) for s in train_dataset['sentence']]

    # Step 2: Instantiate and use a `DictVectorizer`:
    vectorizer = DictVectorizer()
    X = vectorizer.fit_transform(feats)

    # Step 3: Train the model on the feature matrix and
    # train_dataset['gold_label']:
    model.fit(X, train_dataset['gold_label'])

    # Step 4: Return (model, vectorizer):
    return model, vectorizer



You can use the following test to help ensure that your implementation is correct:

In [20]:
def test_train_linear_model(func):
    train_dataset = {
        'sentence': ['A A', 'A B', 'B B', 'B A', 'B'],
        'gold_label': [0, 1, 0, 1, 1]}
    def featfunc(s):
        return Counter(s.split())
    model = LogisticRegression()
    result = func(model, featfunc, train_dataset)
    if not isinstance(result, tuple) or len(result) != 2:
        print(f"Error for `{func.__name__}`: Incorrect return type")
        return
    model, vectorizer = result
    if not hasattr(vectorizer, 'vocabulary_'):
        print(f"Error for `{func.__name__}`: "
              f"Second return value is not a trained vectorizer")
        return
    if not hasattr(model, 'classes_'):
        print(f"Error for `{func.__name__}`: "
              f"First return value is not a trained classifier")
        return
    print(f"No errors found for `{func.__name__}`")

In [21]:
_ = test_train_linear_model(train_linear_model)

No errors found for `train_linear_model`


You can now very easily train models on our datasets. Quick example (this shouldn't take more than a couple of minutes to run even on a CPU):

In [22]:
lr_unigrams, vec_unigrams = train_linear_model(
    LogisticRegression(max_iter=1000),
    unigrams_phi, dynasent_r1['train'])

### Task 3: Model assessment [1 point]

Having now trained a model, we'd like to perform assessments on new data. Your task is to complete the wrapper function `assess_linear_model` to do this. The primary things you need to put into practice are (1) how to use a trained vectorizer on new data and (2) how to make predictions with your trained model. (Both of these steps are reviewed earlier in this notebook.)

In [23]:
def assess_linear_model(model, featfunc, vectorizer, assess_dataset):
    """Assess a trained sklearn model.

    Parameters
    ----------
    model: trained sklearn model
    featfunc : func
        Maps strings to count dicts
    vectorizer : fitted DictVectorizer
    assess_dataset: dict
        Must have a key "sentence" containing strings that `featfunc`
        will process, and a key "gold_label" giving labels

    Returns
    -------
    A classification report (multiline string)

    """

    # Step 1: Featurize the assessment data:
    feats = [featfunc(s) for s in assess_dataset['sentence']]

    # Step 2: Vectorize the assessment data features:
    X = vectorizer.transform(feats)

    # Step 3: Make predictions:
    predictions = model.predict(X)

    # Step 4: Return a classification report (str):
    return classification_report(assess_dataset['gold_label'], predictions, zero_division=0)


Here's a quick test you can use:

In [24]:
def test_assess_linear_model(assessfunc, trainfunc):
    train_dataset = {
        'sentence': ['A A', 'A B', 'B B', 'B A', 'A', 'B'],
        'gold_label': [0, 1, 0, 1, 0, 1]}
    assess_dataset = {
        'sentence': ['A C', 'B A'],
        'gold_label': [0, 1]}
    def featfunc(s):
        return Counter(s.split())
    model = LogisticRegression()
    model, vectorizer = trainfunc(model, featfunc, train_dataset)
    result = assessfunc(model, featfunc, vectorizer, assess_dataset)
    errcount = 0
    if len(vectorizer.vocabulary_) != 2:
        print(f"Error for `{assessfunc.__name__}`: Unexpected feature count")
        errcount += 1
    if 'weighted avg' not in result:
        print(f"Error for `{assessfunc.__name__}`: Unexpected return value")
        errcount += 1
    if errcount == 0:
        print(f"No errors found for `{assessfunc.__name__}`")

In [25]:
test_assess_linear_model(assess_linear_model, train_linear_model)

No errors found for `assess_linear_model`


If you trained a model `lr_unigrams` above, you can now easily assess it. An example:

In [26]:
report = assess_linear_model(
    lr_unigrams,
    unigrams_phi,
    vec_unigrams,
    dynasent_r1['validation'])

print(report)

              precision    recall  f1-score   support

    negative       0.76      0.36      0.49      1200
     neutral       0.52      0.89      0.66      1200
    positive       0.70      0.57      0.63      1200

    accuracy                           0.61      3600
   macro avg       0.66      0.61      0.59      3600
weighted avg       0.66      0.61      0.59      3600



## Question 2: Transformer fine-tuning

We're now going to move into a more modern mode: fine-tuning pretrained components.

We'll use BERT-mini (originally from [the BERT repo](https://github.com/google-research/bert)) for the homework so that we can rapdily develop prototypes. You can then consider scaling up to larger models.

In [30]:
import transformers
from transformers import AutoModel, AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

The `transformers` library does a lot of logging. To avoid ending up with a cluttered notebook, I am changing the logging level. You might want to skip this as you scale up to building production systems, since the logging is very good – it gives you a lot of insights into what the models and code are doing.

In [31]:
transformers.logging.set_verbosity_error()

Here we set ourselves up to use BERT-mini:

In [32]:
weights_name = "prajjwal1/bert-mini"

bert = AutoModel.from_pretrained(weights_name)

bert_tokenizer = AutoTokenizer.from_pretrained(weights_name)

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

### Task 1: Batch tokenization [1 point]

Your task here is to use the `batch_encode_plus` method for `bert_tokenizer` to tokenize a list of strings. You should complete `get_batch_token_ids` according to the specification in the doctring. All these steps can be handled with a single call to `batch_encode_plus`.

In [33]:
def get_batch_token_ids(batch, tokenizer):
    """Map `batch` to a tensor of ids. The return
    value should meet the following specification:

    1. The max length should be 512.
    2. Examples longer than the max length should be truncated
    3. Examples should be padded to the max length for the batch.
    4. The special [CLS] should be added to the start and the special
       token [SEP] should be added to the end.
    5. The attention mask should be returned
    6. The return value of each component should be a tensor.

    Parameters
    ----------
    batch: list of str
    tokenizer: Hugging Face tokenizer

    Returns
    -------
    dict with at least "input_ids" and "attention_mask" as keys,
    each with Tensor values

    """
   # Use batch_encode_plus with the specified parameters:
    encoded_batch = tokenizer.batch_encode_plus(
        batch,
        add_special_tokens=True,  # Add [CLS] and [SEP]
        padding='max_length',     # Pad to max_length
        truncation=True,          # Truncate if longer than max_length
        max_length=512,           # Set max_length to 512
        return_attention_mask=True, # Return attention mask
        return_tensors='pt'       # Return PyTorch tensors
    )
    return encoded_batch




Here's a test you can use:

In [34]:
def test_get_batch_token_ids(func):
    examples = [
        "Bert knows Snuffleupagus",
        "ELMo knew Bert.",
        "Buffalo " * 520
    ]
    test_tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
    result = func(examples, test_tokenizer)
    errcount = 0
    if 'attention_mask' not in result:
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Attention mask was not returned")
    ids = result['input_ids']
    if not isinstance(ids, torch.Tensor):
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Return values are not tensors")
    if ids.shape[1] != 512:
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Expected sequence length 512; got {ids.shape[1]}")
    if ids[0][0] != bert_tokenizer.cls_token_id:
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Special tokens were not added")
    if errcount == 0:
        print(f"No errors found for `{func.__name__}`")

In [35]:
test_get_batch_token_ids(get_batch_token_ids)

No errors found for `get_batch_token_ids`


### Task 2: Contextual representations [1 point]

This next task is not used directly in fine-tuning, but it should help ensure that you understand how BERT representations are created and how they need to be managed.

Your task is to complete `get_reps` so that, given a dataset (list of strings), it returns a single tensor in which each row is the output hidden state above the [CLS] token for that example. `gets_reps` has a batchsize argument that the user can manage depending on how much available memory they have and how large their model is.

In [36]:
def get_reps(dataset, model, tokenizer, batchsize=20):
    """Represent each example in `dataset` with the final hidden state
    above the [CLS] token.

    Parameters
    ----------
    dataset : list of str
    model : BertModel
    tokenizer : BertTokenizerFast
    batchsize : int

    Returns
    -------
    torch.Tensor with shape `(n_examples, dim)` where `dim` is the
    dimensionality of the representations for `model`

    """
    data = []
    with torch.no_grad():
        for i in range(0, len(dataset), batchsize):
            batch = dataset[i : i + batchsize]  # Get current batch

            # Encode the batch using get_batch_token_ids:
            encoded_batch = get_batch_token_ids(batch, tokenizer)

            # Get representations from the model:
            output = model(**encoded_batch)

            # Extract the hidden state above the [CLS] token:
            batch_reps = output.last_hidden_state[:, 0, :]
            data.append(batch_reps)

    # Concatenate all batch representations into a single tensor:
    return torch.cat(data, dim=0)




Quick test:

In [37]:
def test_get_reps(func):
    examples = ["The cat slept.", "The bird chirped."] * 20
    weights_name = "prajjwal1/bert-mini"
    test_model = AutoModel.from_pretrained(weights_name)
    test_tokenizer = AutoTokenizer.from_pretrained(weights_name)
    result = func(examples, test_model, test_tokenizer, batchsize=2)
    errcount = 0
    if result.shape != (40, 256):
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Expected shape {(40, 256)}, got {result.shape}")
    if round(result[0][0].item(), 2) != -0.64:
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Representations seem to be incorrect")
    if errcount == 0:
        print(f"No errors found for `{func.__name__}`")

In [38]:
test_get_reps(get_reps)

No errors found for `get_reps`


### Task 3: Fine-tuning module [1 point]

We can now put the above together into a basic `nn.Module` that will fine-tune our BERT model. Most of the module is written for you. The pieces you need to implement:

1. in the `init` methid, define `self.classifier_layer` using [nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
2. Complete the `forward` method.

Precise instructions are provided in the docstrings for the model.

In [39]:
import torch.nn as nn

class BertClassifierModule(nn.Module):
    def __init__(self,
            n_classes,
            hidden_activation,
            weights_name="prajjwal1/bert-mini"):
        """This module loads a Transformer based on  `weights_name`,
        puts it in train mode, add a dense layer with activation
        function give by `hidden_activation`, and puts a classifier
        layer on top of that as the final output. The output of
        the dense layer should have the same dimensionality as the
        model input.

        Parameters
        ----------
        n_classes : int
            Number of classes for the output layer
        hidden_activation : torch activation function
            e.g., nn.Tanh()
        weights_name : str
            Name of pretrained model to load from Hugging Face

        """
        super().__init__()
        self.n_classes = n_classes
        self.weights_name = weights_name
        self.bert = AutoModel.from_pretrained(self.weights_name)
        self.bert.train()
        self.hidden_activation = hidden_activation
        self.hidden_dim = self.bert.embeddings.word_embeddings.embedding_dim
         # Define self.classifier_layer using nn.Sequential
        self.classifier_layer = nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes)
        )



    def forward(self, indices, mask):
        """Process `indices` with `mask` by feeding these arguments
        to `self.bert` and then feeding the initial hidden state
        in `last_hidden_state` to `self.classifier_layer`

        Parameters
        ----------
        indices : tensor.LongTensor of shape (n_batch, k)
            Indices into the `self.bert` embedding layer. `n_batch` is
            the number of examples and `k` is the sequence length for
            this batch
        mask : tensor.LongTensor of shape (n_batch, d)
            Binary vector indicating which values should be masked.
            `n_batch` is the number of examples and `k` is the
            sequence length for this batch

        Returns
        -------
        tensor.FloatTensor
            Predicted values, shape `(n_batch, self.n_classes)`

        """
         # Get BERT representations
        output = self.bert(indices, attention_mask=mask)

        # Extract hidden state above [CLS] token
        hidden_state = output.last_hidden_state[:, 0, :]

        # Pass hidden state through classifier layer
        logits = self.classifier_layer(hidden_state)

        return logits



In [40]:
bert_module = BertClassifierModule(n_classes=3, hidden_activation=nn.Tanh())

In [41]:
ids = get_batch_token_ids(
    dynasent_r1['train']['sentence'][: 2],
    bert_tokenizer)

bert_module(ids['input_ids'], ids['attention_mask'])

tensor([[ 0.1852,  0.2985, -0.0096],
        [ 0.3733,  0.2044, -0.2236]], grad_fn=<AddmmBackward0>)

In [42]:
def test_bert_classifier_module(moduleclass):
    expected_out = 5
    expected_hidden = 256
    expected_activation = nn.ReLU()
    mod = moduleclass(expected_out, expected_activation)
    errcount = 0

    # Basic layer structure:
    if not hasattr(mod, "classifier_layer") or mod.classifier_layer is None:
        errcount += 1
        print(f"Error for `{moduleclass.__name__}`: "
              f"Missing attribute `classifier_layer`")
        return
    for i in range(3):
        try:
            bert_module.classifier_layer[i]
        except IndexError:
            errcount += 1
            print(f"Error for `{moduleclass.__name__}`: "
                  f"`classifier_layer` is not an `nn.Sequential` "
                  f"and/or does not have the right structure")
    # Correct first layer dimensionality:
    result_hidden = mod.classifier_layer[0].out_features
    if result_hidden != expected_hidden:
        errcount += 1
        print(f"Error for `{moduleclass.__name__}`: "
              f"Expected `classifier_layer` hidden dim {expected_hidden}, "
              f"got {result_hidden}")
    # Correct activation:
    result_activation = mod.classifier_layer[1].__class__.__name__
    if result_activation != expected_activation.__class__.__name__:
        errcount += 1
        print(f"Error for `{moduleclass.__name__}`: "
              f"Incorrect hidden activation")
    # Correct output dimensionality:
    result_out = mod.classifier_layer[2].out_features
    if result_out != expected_out:
        errcount += 1
        print(f"Error for `{moduleclass.__name__}`: "
              f"Expected `classifier_layer` out dim {expected_out}, "
              f"got {result_out}")
    # forward method:
    ids = get_batch_token_ids(["A B C", "A B"], bert_tokenizer)
    result = mod(ids['input_ids'], ids['attention_mask'])
    if result.shape != (2, 5):
        errcount += 1
        print(f"Error for `{moduleclass.__name__}`: "
              f"Expected output shape {(2, 5)}, got {result.shape}")
    if errcount == 0:
        print(f"No errors found for `{moduleclass.__name__}`")

In [43]:
test_bert_classifier_module(BertClassifierModule)

No errors found for `BertClassifierModule`


## Question 3: Your original system [3 points]

Your task is to develop an original ternary sentiment classifier model. There are many options. The only rule:

__You cannot make any use of the test sets for DynaSent-R1, DynaSent-R2, or SST-3, at any time during the course of development.__

The integrity of the bakeoff depends on this rule being followed.

It's fine to use the dev sets for system development – indeed, we encourage this.

For system development, here are some relatively manageable ideas that you might try:

* Different pretrained models. There are many models available on the [Hugging Face models hub](https://huggingface.co/models) that will be drop-in replacements for BERT-mini as we used it above.

* Different fine-tuning regimes. We used the [CLS] token above. This doesn't make especially good use of the output states of the models. Pooling across these representtions (with sum, average, etc.) is likely to be better.

* Different training regimes. You have three train sets at your disposal, and there may be other sentiment datasets that could contribute to making your system more robust in new domains.

* Entirely different approaches. There is no requirement that you make use of any of the concepts from the homework questions in constructing your original system. Anything goes as long as you follow the one rule given above in bold.

We want to emphasize that this needs to be an original system. It doesn't suffice to download code from the Web, retrain, and submit. You can build on others' code, but you have to do something new and meaningful with it. See the course website for additional guidance on how original systems will be evaluated.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# START COMMENT: Enter your system description in this cell.

def foo(s):
    return True

# STOP COMMENT: Please do not remove this comment.

## Question 4: Bakeoff entry [1 point]

The bakeoff dataset is available at

https://web.stanford.edu/class/cs224u/data/cs224u-sentiment-test-unlabeled.csv

This code should grab it for you and put it in `data/sentiment` if you are working in the cloud:

In [ ]:
import os
import wget

if not os.path.exists(os.path.join("data", "sentiment", "cs224u-sentiment-test-unlabeled.csv")):
    os.makedirs(os.path.join('data', 'sentiment'), exist_ok=True)
    wget.download('https://web.stanford.edu/class/cs224u/data/cs224u-sentiment-test-unlabeled.csv', out='data/sentiment/')

If the above fails, you can just download the file and place it in `data/sentiment`.

Once you have the file, you can load it to a `pd.DataFrame`:

In [ ]:
bakeoff_df = pd.read_csv(
    os.path.join("data", "sentiment", "cs224u-sentiment-test-unlabeled.csv"))

In [ ]:
bakeoff_df.head()

To enter the bakeoff, you simply need to use your original system to:

1. Add a column named 'prediction' to `cs224u-sentiment-test-unlabeled.csv` with your model predictions (which are strings in {`positive`, `negative`, `neutral`}). The existing columns should remain.

2. Save the file as `cs224u-sentiment-bakeoff-entry.csv`. Here is a good snippet of code for writing this file:

In [ ]:
# This is a placeholder for adding the "prediction" column:
# bakeoff_df['prediction'] = # Use your model to add predictions.

# Write to disk
bakeoff_df.to_csv("cs224u-sentiment-bakeoff-entry.csv")

In particular, you need to be sure that `example_id` is a column rather than an index when read in by Pandas. Here is a quick test:

In [ ]:
def test_bakeoff_entry(filename="cs224u-sentiment-bakeoff-entry.csv"):
    gold_df = pd.read_csv(
        os.path.join("data", "sentiment", "cs224u-sentiment-test-unlabeled.csv"))
    entry_df = pd.read_csv(filename)

    # Check that no required columns are missing:
    expected_cols = {'example_id', 'sentence', 'prediction'}
    missing_cols = expected_cols - set(entry_df.columns)
    errcount = 0
    if len(missing_cols) != 0:
        errcount += 1
        print(f"Entry is missing required columns {missing_cols}")
        return

    # Check that the predictions are in our space:
    labels = {'positive', 'negative', 'neutral'}
    predtypes = set(entry_df.prediction.unique())
    unexpected = predtypes - labels
    if len(unexpected) != 0:
        errcount += 1
        print(f"Prediction column has unexpected values: {unexpected}")

    # Check that the dataset hasn't been rearranged:
    for colname in ('example_id', 'sentence'):
        if not entry_df[colname].equals(gold_df[colname]):
            errcount += 1
            print(f"Entry is misaligned with test data on column {colname}")

    # Clean bill of health:
    if errcount == 0:
        print("No errors detected with `test_bakeoff_entry`.")

In [ ]:
test_bakeoff_entry("cs224u-sentiment-bakeoff-entry.csv")

Submit the following files to Gradescope:

* `hw_sentiment.ipynb` (this notebook)
* `cs224u-sentiment-bakeoff-entry.csv` (bake-off output)

Please make sure you use these filenames. The autograder looks for files with these names.

You are not permitted to do any tuning of your system based on what you see in our bakeoff prediction file – you should not study that file in anyway, beyond perhaps checking that it contains what you expected it to contain. The upload function will do some additional checking to ensure that your file is well-formed.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points.